# Summary analysis: Skype vs fixed trajectory
## 4 experiments, 2 animals each responding to black disc moving on various paths

In [ ]:
%config InteractiveShellApp.pylab_import_all = False
%matplotlib inline
%pylab inline
%reload_ext autoreload
%autoreload 2

import sys
import os
import fnmatch

import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame, Series
import seaborn as sns

propsFn='props.csv'
props=pd.read_csv(propsFn, header=None, index_col=0, squeeze=True,delim_whitespace=True).to_dict()

base=props['BaseDir']
expFile=props['allExpFn']

RawDataDir = os.path.join(base,props['RawDataDir'])+'\\'
ProcessingDir = os.path.join(base,props['ProcessingDir'])+'\\FigS1b\\'
outputDir = os.path.join(base,props['outputDir'])+'\\'

if not os.path.isdir(ProcessingDir):
    os.makedirs(ProcessingDir)
if not os.path.isdir(outputDir):
    os.makedirs(outputDir)

os.chdir('..\\')

import functions.matrixUtilities_joh as mu
import matplotlib.pyplot as plt
import models.experiment as xp
import models.experiment_set as es
import functions.paperFigureProps as pfp
pfp.paper()
inToCm=2.54


Pre-Analyze all experiments only if necessary, this takes a couple of minutes! Experiment summary csv files are saved to disk.

In [ ]:
info=pd.read_csv(expFile, sep=',')#pd.read_csv(expFile,quotechar='"', sep=',', converters={'bdGroup':ast.literal_eval})
info=info[info.stimulusProtocol=='stacks']
info

In [ ]:
posPath = []
aviPath=[]
bdGroupAll=[]
bgAll=[]
expTimeAll=[]

anSet=[1,0,0,1,0,1,1,0,0,0,0,0,0]

for index,row in info.iterrows():
    currTime=datetime.strptime(row.date, '%Y%m%d%H%M%S')
    DishDir=RawDataDir+row.path
    #DishDir='D:\\data\\b\\2016\\'+row.path
    

    for root, dirnames, filenames in os.walk(DishDir):
        for filename in fnmatch.filter(filenames, '*nogaps*.txt'):
            posPath.append(os.path.join(root, filename))
            bdGroupAll.append('0 0')
            bgAll.append(row.bd)
            #currTime=datetime.strptime(filename[-23:-4], '%Y-%m-%dT%H_%M_%S')
            expTimeAll.append(currTime)
        for filename in fnmatch.filter(filenames, '*split*.avi'):
            aviPath.append(os.path.join(root, filename))


infoPAll=pd.DataFrame({'txtPath': posPath})
infoPAll['epiDur']=60
infoPAll['bd']=bgAll
infoPAll['bdGroup']=bdGroupAll
infoPAll['episodes']=-1
infoPAll['inDish']=10
#info['stimulusProtocol']=8
infoPAll['arenaDiameter_mm'] = 100 # arena diameter (default: 100 mm)
infoPAll['minShift'] = 60 # minimum number of seconds to shift for control IAD
infoPAll['episodePLcode'] = 0 # flag if first two characters of episode name encode animal pair matrix (default: 0)
infoPAll['recomputeAnimalSize'] = 0 # flag to compute animals size from avi file (takes time, default: 1)
infoPAll['SaveNeighborhoodMaps'] = 0 # flag to save neighborhood maps for subsequent analysis (takes time, default: 1)
infoPAll['computeLeadership'] = 0 # flag to compute leadership index (takes time, default: 1)
infoPAll['ComputeBouts'] = 0 # flag to compute swim bout frequency (takes time, default: 1)
infoPAll['set'] = anSet   # experiment set: can label groups of experiments (default: 0)
infoPAll['expTime']=expTimeAll
infoPAll['pxPmm']=8
infoPAll['ProcessingDir']=ProcessingDir
infoPAll['outputDir']=outputDir
infoPAll['allowEpisodeSwitch']=1



csvFile=os.path.join(ProcessingDir,'FigS1b.csv')
infoPAll.to_csv(csvFile,encoding='utf-8')

infoPAll

In [ ]:
def readExperiment(keepData=False):
    tmp=es.experiment_set(csvFile=csvFile)
    if keepData:
        return tmp
    else:
        return 1

expSet=readExperiment(keepData=True)

In [ ]:
csvPath = []
for f in [mu.splitall(x)[-1][:-4] for x in infoPAll.txtPath]:
    csvPath.append(glob.glob(ProcessingDir+f+'*siSummary*.csv')[0])


df=pd.DataFrame()
i=0
for fn in csvPath:
    print(fn)
    tmp=pd.read_csv(fn,index_col=0,sep=',')
    #tmp.animalSet=i
    tmp['animalIndexCont']=tmp.animalIndex+((i)*15)
    df=pd.concat([df,tmp])
    i+=1

df=df[df['animalIndex'].isin(np.arange(0,14,2))]
#df['episode']='skype'
print('df shape',df.shape)

d=df.time
r=datetime(2016,1,1)
t2=[pd.to_datetime(x).replace(day=1,month=1)for x in df.time]

t3=[(x-r)/pd.Timedelta('1 hour') for x in t2]
df['t2']=t2
df['t3']=t3
df.head()

In [ ]:
epiNames=['C','Stack']
df['epiNames']=[epiNames[x] for x in df.animalSet.values]
df.sort_values(by='animalSet',inplace=True)
df

In [ ]:
pfp.paper()
inToCm=2.54
plt.figure(figsize=(2/inToCm,4.5/inToCm))
ax = plt.gca()
col1=['gray','k']

sns.swarmplot(x='epiNames',
              y='si',
              data=df,
              linewidth=1,
            palette=col1,
              zorder=0)

sns.pointplot(data=df,x='epiNames',y='si',ax=ax,palette=['k'],zorder=100,markers=['_'],scale=3,errwidth=1,capsize=.3)
sns.pointplot(data=df,x='epiNames',y='si',ax=ax,palette=['r'],zorder=100,markers=['_'],scale=3,ci=None)

#sns.barplot(y='si',x='epiNames',data=df)
plt.ylim([0,1])
plt.yticks(np.arange(0,1.1,.5))

plt.ylabel('Attraction')
#for tick in ax.xaxis.get_major_ticks():
#    tick.label.set_rotation(45)
plt.xlabel('')
sns.despine()
figPath=outputDir+'\\S1b_DishVsStack.svg'
plt.savefig(figPath)

from shutil import copy2

def splitall(path):
    allparts = []
    while 1:
        parts = os.path.split(path)
        if parts[0] == path:  # sentinel for absolute paths
            allparts.insert(0, parts[0])
            break
        elif parts[1] == path: # sentinel for relative paths
            allparts.insert(0, parts[1])
            break
        else:
            path = parts[0]
            allparts.insert(0, parts[1])
    return allparts


for i,row in infoPAll.iterrows():
    fn=row.txtPath
    head, tail = os.path.split(fn)

    copyList=[]
    
    copyList.append(glob.glob(head+'\\*nogaps.txt')[0])
    copyList.append(glob.glob(head+'\\*animalSize.txt')[0])
    #aviTime=glob.glob(head+'\\*.avi')[0][-18:-4]
    aviTime=row.expTime#datetime.strptime(aviTime,'%Y%m%d%H%M%S')


    for f in copyList:
        print(f)
        if f[0]=='E':
            keepSlash=3
        else:
            keepSlash=4
        toDirectory = "e:\\b\\LarschAndBaier2018\\RawData\\" + os.path.join(*splitall(f)[keepSlash:-1])+"\\"
        #toDirectory = "e:\\b\\LarschAndBaier2018\\RawData\\" 
        if not os.path.isdir(toDirectory):
            os.makedirs(toDirectory)
        
        copy2(f, toDirectory)
        #os.chdir(toDirectory)
        if 'nogaps.txt' in f:
            
            old=glob.glob(toDirectory+'\\*nogaps.txt')[0]
            #t=datetime.strftime(row.expTime, '%Y-%m-%dT%H_%M_%S')
            t=datetime.strftime(aviTime, '%Y-%m-%dT%H_%M_%S')
            #t=aviTime#, '%Y%m%d%H%M%S'
            new=old[:-4]+str(i).zfill(2)+"_"+t+'.txt'
            os.rename(old,new)
            print(new)